## Training

In [56]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
import random as rd 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [57]:
df=pd.read_csv(r'C:\Users\asus\Documents\Projects\App_Subscription_Analyser\Dataset\PreProcessedData.csv',index_col=False)
df.head(4)


,enrolled,VerifyPhone,Other,VerifyDateOfBirth,location,is_weekend_enrolleddate,VerifyCountry,Credit,numscreens,BankVerification,idscreen,VerifyMobile,Loan_all,Alerts,age
0,0,1,7,1,0,0,0,0,15,0,1,0,1,0,23
1,0,1,5,1,1,0,1,0,13,0,0,0,1,0,24
2,0,0,0,0,0,0,0,0,3,0,0,0,1,0,23
3,1,1,6,1,1,0,0,3,40,0,0,0,1,0,28


In [58]:
df['enrolled'].value_counts()
#The data is pretty much balanced

enrolled
0    25213
1    24787
Name: count, dtype: int64

In [59]:
X=df.drop('enrolled',axis=1)
Y=df['enrolled']

In [60]:
scale=StandardScaler()
X=scale.fit_transform(X)


In [61]:
df.shape

(50000, 15)

In [62]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.8,stratify=Y)

#### Using random prediction

In [63]:
def random_pred(X,Y):
    res=[]
    for i in range(0,len(Y)):
        res.append(rd.choice([0,1]))
    return res


In [64]:
Y_random_pred=random_pred(X_train,Y_train)
rand_train_score=accuracy_score(Y_train,Y_random_pred)*100
rand_train_score

49.45

In [65]:
Y_random_test_pred=random_pred(X_test,Y_test)
rand_test_score=accuracy_score(Y_test,Y_random_test_pred)*100
rand_test_score

49.487500000000004

### Using logistic regression

In [66]:
log_reg=LogisticRegression()
log_reg.fit(X_train,Y_train)


LogisticRegression()

In [67]:
Y_pred=log_reg.predict(X_train)
log_reg_train_score=accuracy_score(Y_train,Y_pred)*100
log_reg_train_score

78.07

In [68]:
Y_test_pred=log_reg.predict(X_test)
log_reg_test_score=accuracy_score(Y_test,Y_test_pred)*100
log_reg_test_score

78.375

### Using SVM

In [69]:
svc=SVC()
svc.fit(X_train,Y_train)


SVC()

In [70]:
Y_train_pred_svc=svc.predict(X_train)
svc_train_score=accuracy_score(Y_train,Y_train_pred_svc)*100
svc_train_score

80.01

In [71]:
Y_test_pred_svc=svc.predict(X_test)
svc_test_score=accuracy_score(Y_test,Y_test_pred_svc)*100
svc_test_score

79.335

### Using Decision Tree

In [72]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,Y_train)

DecisionTreeClassifier()

In [73]:
Y_train_pred_dtc=dtc.predict(X_train)
dtc_train_score=accuracy_score(Y_train,Y_train_pred_dtc)*100
dtc_train_score

99.15

In [74]:
Y_test_pred_dtc=dtc.predict(X_test)
dtc_test_score=accuracy_score(Y_test,Y_test_pred_dtc)*100
dtc_test_score

70.8125

### Using Random Classifier

In [75]:
rfc=RandomForestClassifier()
rfc.fit(X_train,Y_train)

RandomForestClassifier()

In [76]:
Y_train_pred_rfc=rfc.predict(X_train)
rf_train_score=accuracy_score(Y_train,Y_train_pred_rfc)*100
rf_train_score

99.15

In [77]:
Y_test_pred_rfc=rfc.predict(X_test)
rf_test_score=accuracy_score(Y_test,Y_test_pred_rfc)*100
rf_test_score

77.2325

#### Scoreboard and observation

In [78]:
algo=["Random","Logistic Regression","SVM","Decision Tree","Random Forest"]
train_score=[rand_train_score,log_reg_train_score,svc_train_score,dtc_train_score,rf_train_score]
test_score=[rand_test_score,log_reg_test_score,svc_test_score,dtc_test_score,rf_test_score]
remark=["Random predictor","Good working","Score improved and best fit","Overfit","Overfit"]
tab={"Algorithm":algo,"Train Score":train_score,"Test Score":test_score,"Remarks":remark}
scoreboard=pd.DataFrame(tab)
scoreboard

,Algorithm,Train Score,Test Score,Remarks
0,Random,49.45,49.4875,Random predictor
1,Logistic Regression,78.07,78.3750,Good working
2,SVM,80.01,79.3350,Score improved and best fit
3,Decision Tree,99.15,70.8125,Overfit
4,Random Forest,99.15,77.2325,Overfit
